In [1]:
%matplotlib inline
import SemDiff as sd
import pandas as pd
from gensim.models import Word2Vec

In [2]:
# I'm assuming that the models are already saved and accessible
intro_model = Word2Vec.load("../data/intro_citance_model.wv")
methods_model = Word2Vec.load("../data/methods_citance_model.wv")
discussion_model = Word2Vec.load("../data/discussion_citance_model.wv")

In [3]:
sd.project_word_on_axis(intro_model, "theory", ["certain", "uncertain"], k = 3)

0.24428386986255646

In [4]:
sd.project_word_on_axis(methods_model, "theory", ["certain", "uncertain"], k = 3)

0.07503204047679901

In [5]:
# repeat for a set of antonyms
antonyms = [
    ["good", "bad"],
    ['always', 'never'],
    ['higher', 'lower'],
    ['true', 'false'],
    ['certain', 'impossible'],
    ['strong', 'weak'],
    ['very', 'slightly'],
    ['similar', 'unlike'],
    ['arbitrary', 'reasonable'],
    ['unexpected', 'expected'],
    ['outside', 'inside'],
    ['hypothetical', 'actual'],
    ['under', 'upper'],
    ['systemic', 'endemic'],
    ['new', 'established'],
    ['known', 'unknown'],
    ['propose', 'answer'],
    ['start', 'stop'],
    ['worst', 'best']
]

words = ['result', 'study', 'method', 'theory', 'literature', 'finding', 'approach']
intro_list = []
methods_list = []
discussion_list = []
ant_names = []
word_list = []
for word in words:
    for ant in antonyms:
        intro_list.append(sd.project_word_on_axis(intro_model, word, ant, k = 3))
        methods_list.append(sd.project_word_on_axis(methods_model, word, ant, k = 3))
        discussion_list.append(sd.project_word_on_axis(discussion_model, word, ant, k = 3))
        word_list.append(word)
    
        ant_names.append("{}-{}".format(ant[0], ant[1]))

/Users/dakotamurray/.pyenv/versions/3.7.1/lib/python3.7/site-packages/scipy/spatial/distance.py:702: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [6]:
df = pd.DataFrame({'antonym': ant_names, 
                   'intro': intro_list, 
                   'methods': methods_list, 
                   'discussion': discussion_list,
                   'word': word_list}, 
                  index = ant_names)
df.head()

,antonym,intro,methods,discussion,word
good-bad,good-bad,0.167759,0.017149,0.079017,result
always-never,always-never,-0.065391,-0.285983,-0.249468,result
higher-lower,higher-lower,0.095719,0.027025,NaN,result
true-false,true-false,-0.095423,-0.010417,0.037423,result
certain-impossible,certain-impossible,-0.023111,-0.121604,-0.110451,result


In [7]:
df.to_csv('../data/semantic_word_axes.csv')

Then, we can remake the data that was used in the ISSI submission, but with some new words

In [8]:
shared = sd.get_shared_words(intro_model, methods_model, topn_words = 10000)
len(shared)

6879

In [9]:
words = ['result', 'study', 'method', 'theory', 'literature', 'finding', 'approach', 'observation', 'significant']
frames = []
for word in words:
    frames.append(sd.model_term_similarity(intro_model, methods_model, word, shared))
    
intro_meth_df = pd.concat(frames)
intro_meth_df.shape

(61911, 3)

In [10]:
intro_meth_df.to_csv('../data/intro_method_interword_similarity.csv')

In [11]:
shared = sd.get_shared_words(intro_model, methods_model, topn_words = 10000)
len(shared)

frames = []
for word in words:
    frames.append(sd.model_term_similarity(intro_model, discussion_model, word, shared))
    
intro_disc_df = pd.concat(frames)
intro_disc_df.shape

(60183, 3)

In [12]:
intro_disc_df.to_csv('../data/intro_discussion_interword_similarity.csv')